>原文：
- https://blog.csdn.net/xuanwolanxue/article/details/62236413?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task

> ```
> 注：本篇笔记以gstreamer 1.8.1 版本为蓝本，若有出入的地方，请确认版本信息
> ```

# 1. log 调试信息的函数

在 gstreamer 1.8.1 版本中定义了如下的 log 输出宏:

## 1.1 最基本的 log 输出宏

```c
GST_CAT_LEVEL_LOG(cat,level,object,...)
```

其中

*   cat：表示 category，用于区分 log 信息的类别，在后面说到 debug level 等信息的设置的时候会用到；
*   level：表示当前输出的这条 log 信息属于哪一个等级，具体的等级定义，将在后文中介绍；
*   obj：表示输出的 log 信息属于哪一个 GObject，注：此项可以设置为 NULL，表示不属于任何 GObject；
*   … : 为 log 信息的内容。

## 1.2 不同的 log 输出宏分组

为了使用的方便，根据不同的输出需求，在基础输出宏的基础上再做了一次封装，同样是使用宏定义的方式。

*   GST_CAT_ERROR_OBJECT(cat,obj,…)
*   GST_CAT_WARNING_OBJECT(cat,obj,…)
*   GST_CAT_INFO_OBJECT(cat,obj,…)
*   GST_CAT_DEBUG_OBJECT(cat,obj,…)
*   GST_CAT_LOG_OBJECT(cat,obj,…)
*   GST_CAT_FIXME_OBJECT(cat,obj,…)
*   GST_CAT_TRACE_OBJECT(cat,obj,…)

这一组主要使用到了 category 和 GObject

*   GST_CAT_ERROR(cat,…)
*   GST_CAT_WARNING(cat,…)
*   GST_CAT_INFO(cat,…)
*   GST_CAT_DEBUG(cat,…)
*   GST_CAT_LOG(cat,…)
*   GST_CAT_FIXME(cat,…)
*   GST_CAT_TRACE(cat,…)

这一组只使用到了 category，而 obj 项将被设置为 NULL

*   GST_ERROR_OBJECT(obj,…)
*   GST_WARNING_OBJECT(obj,…)
*   GST_INFO_OBJECT(obj,…)
*   GST_DEBUG_OBJECT(obj,…)
*   GST_LOG_OBJECT(obj,…)
*   GST_FIXME_OBJECT(obj,…)
*   GST_TRACE_OBJECT(obj,…)

这一组只使用到了 obj， 而 category 项将被设置为 GST_CAT_DEFAULT，即默认类别

*   GST_ERROR(…)
*   GST_WARNING(…)
*   GST_INFO(…)
*   GST_DEBUG(…)
*   GST_LOG(…)
*   GST_FIXME(…)
*   GST_TRACE(…)

这一组最简洁，log 输出时，只需要输出内容，其中 category 为 GST_CAT_DEFAULT，obj 为 NULL。

> 注： 官方建议不要在代码中轻易使用printf或g_print之类的函数，因为他们无法控制是否输出

# 2. 日志等级

在 gstreamer 1.8.1 版本中对 debug level 的定义如下（注：不同的版本中 debug level 的数量可能会有差别）：

```c
typedef enum {
  GST_LEVEL_NONE = 0,
  GST_LEVEL_ERROR = 1,
  GST_LEVEL_WARNING = 2,
  GST_LEVEL_FIXME = 3,
  GST_LEVEL_INFO = 4,
  GST_LEVEL_DEBUG = 5,
  GST_LEVEL_LOG = 6,
  GST_LEVEL_TRACE = 7,
  /* add more */
  GST_LEVEL_MEMDUMP = 9,
  /* add more */
  GST_LEVEL_COUNT
} GstDebugLevel;
```

也就是说，gstreamer1.8.1 当中将 debug level 分为了 10 个等级：

- 0：什么都不打印
- 1：打印 error 的信息
- 2，警告
- 3：需要修复的
- 4，信息(流程性的)
- 5：调试信息
- 6：所有message
- 7：跟踪信息
- 8：堆栈信息
- …

以此类推，其中等级是包含关系，比如设置level为4的话，1,2,3,4类型的信息都会打印。debug level **等级数字越大，输出的 log 信息就越多。**

# 3. 使用命令行参数开启或设置 debug 信息

>gst-launch-1.0 **--gst-debug-level=4** xxxxxxxxxxxx

*   --gst-debug-level=LEVEL  
    其中 LEVEL 的取值为 [0,9], 使用这种方式将同时开启 / 关闭所有模组的 debug 信息，可能会导致 debug 信息过多而影响关键信息的获取
    
*   --gst-debug=STRING  
    其中 STRING 为字符串，使用这种方式只输出由 STRING 指定范围内的调试信息，格式为 ： category_name:LEVEL， 如果要设置多个类别时，每个类别设置之间使用 “,” 分开，例如：–gst-debug="videodecoder:5,fdsrc:4,oggdemux:3"
  
* 更多选项

```  
GStreamer 选项
  --gst-fatal-warnings              将所有的警告也当作严重问题
  --gst-debug-help                  打印可用的调试类别并退出
  --gst-debug-level=级别            默认调试级别，从 1(仅有错误)到 9(所有内容)，0代表无输出
  --gst-debug=列表                  逗号分隔的“类别名称:级别”列表用来为每个类别设置指定的调试级别。如:GST_AUTOPLUG:5,GST_ELEMENT_*:3
  --gst-debug-no-color              禁止彩色的调试信息输出
  --gst-debug-color-mode            改变调试日志的颜色模式。可能的模式有：off、on、disable、auto和unix
  --gst-debug-disable               禁止调试
```   

# 4. 使用环境变量开启或设置 debug 信息


gstreamer 中还可以使用环境变量开启或关闭 debug 信息，  
此环境变量为：`GST_DEBUG`，  
其使用方式和命令行参数 `–gst-debug` 一样，都是有一对对 category_name:LEVEL 这样的二元组构成，二元组与二元组之间使用 “，”（逗号）隔开。

```sh
export GST_DdEBUG=2,audiotestsrc:6
```
It will use Debug Level 6 for the audiotestsrc element, and 2 for all the others.

> 注意: 
>     1. 二元组与二元组之间只能用逗号隔开，其不能有空格等其他符号; 
>     2. category_name支持"*"通配符, 如 "GST_ELEMENT_*:4"

```sh
GST_DEBUG=2,audio*:6
```

will use Debug Level 5 for all categories starting with the word audio. 

```sh
GST_DEBUG=*:2  与 GST_DEBUG=2 等价
```

# 5. 在代码中动态设置 debug 等级

在前面提到的两种开启或设置 debug 信息的方式都是在调用 `gst_init()` 函数期间完成的。也就是说它们有一个缺点，就是无法在运行过程中动态的调整 debug 信息的输出方式与等级，要实现这一个功能就需要通过调用一些函数接口的方式来动态调整 log 信息的输出等级。

*   `gst_debug_set_default_threshold(level)`   
控制全局（所有类别）的 log 等级
    
*   `gst_debug_set_threshold_for_name(name, level)`  
控制某一个类别的 log 等级，如：xxx(“videodecoder”, 5)
    
*  ` gst_debug_set_threshold_from_string(list, reset)`  
和 GST_DEBUG 类似，也是支持同时设置多个元组的 log 信息，后面一个 reset 参数： TRUE：清除之前的 log 输出设置信息，FALSE：将设置信息添加到现有的 log 信息设置当中